In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions anda
# limitations under the License.
# ==============================================================================

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_models_entertainment-with-pretrained-embeddings/nvidia_logo.png" style="width: 90px; float: right;">

# Training with pretrained embeddings

## Overview

In this use case we will consider how we might train with pretrained embeddings.

Pretrained embeddings can allow our model to include information from additional modalities (for instance, we might want to grab CNN descriptors of product images). They can also come from other models that we train on our data. For example, we might train a word2vec model on the sequence of purchased items by a customer and want to include this information in our retrieval or ranking model.

The use cases are many, but this particular example will focus on the technical aspects of working with pretrained embeddings.

We will use a synthetic version of the MovieLens 100k dataset and emulate a scenario where we would have a pretrained embedding for each of the movies in the dataset.

### Learning objectives

- Training with pretrained embeddings
- Understanding [the Schema file](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/schema/schema.py)

## Downloading and preparing the dataset

In [2]:
import merlin.models.tf as mm
from merlin.schema.tags import Tags
import tensorflow as tf
from merlin.models.tf.prediction_tasks.classification import BinaryClassificationTask
from merlin.models.tf.blocks import *
from merlin.datasets.synthetic import generate_data

import numpy as np

2022-08-16 11:37:08.876815: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-16 11:37:10.012480: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16249 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:2d:00.0, compute capability: 7.0


In [3]:
train = generate_data('movielens-100k', num_rows=100_000)

In [4]:
target_column = train.schema.select_by_tag(Tags.TARGET).column_names[1]
target_column

'rating_binary'

The schema holds vital information about our dataset. We can extract the embedding table size for the `moveId` column from it.

In [5]:
train.schema['movieId'].properties['embedding_sizes']['cardinality']

1680.0

From the schema, we can tell that the cardinality of `movieId` is 1680. Index 0 will be used in case an unknown `movieId` is encountered

In order to accommodate this, we initialize our embedding table of dimensionality of (1681, 128).

In [6]:
pretrained_movie_embs = np.random.random((1681, 128))

In [7]:
schema = train.schema

This is only a mock up embedding table. In reality, this is where we would pass our embeddings from another model.

The dimensionality of each embedding, that of 128, is arbitrary. We could have specified some other value here, though generally multiples of 8 tend to work well.

We need to update the schema properties of our `movieId` column since we will not be using the default embedding dimension.

In [8]:
schema['movieId'].properties['embedding_sizes'] = {
    'cardinality': float(pretrained_movie_embs.shape[0]), 
    'dimension': float(pretrained_movie_embs.shape[1])
}

In [9]:
train.schema = schema

In [10]:
train.schema

,name,tags,dtype,is_list,is_ragged,properties.freq_threshold,properties.max_size,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.cat_path,properties.num_buckets,properties.start_index,properties.domain.min,properties.domain.max
0,movieId,"(Tags.ITEM_ID, Tags.CATEGORICAL, Tags.ITEM)",int32,False,False,0.0,0.0,1681.0,128.0,.//categories/unique.movieId.parquet,NaN,0.0,0.0,1680.0
1,userId,"(Tags.USER, Tags.CATEGORICAL, Tags.USER_ID)",int32,False,False,0.0,0.0,943.0,74.0,.//categories/unique.userId.parquet,NaN,0.0,0.0,943.0
2,genres,"(Tags.CATEGORICAL, Tags.ITEM)",int32,False,False,0.0,0.0,216.0,32.0,.//categories/unique.genres.parquet,NaN,0.0,0.0,216.0
3,TE_movieId_rating,(Tags.CONTINUOUS),float64,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,userId_count,(Tags.CONTINUOUS),float32,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,gender,"(Tags.USER, Tags.CATEGORICAL)",int32,False,False,0.0,0.0,2.0,16.0,.//categories/unique.gender.parquet,NaN,0.0,0.0,2.0
6,zip_code,"(Tags.USER, Tags.CATEGORICAL)",int32,False,False,0.0,0.0,795.0,67.0,.//categories/unique.zip_code.parquet,NaN,0.0,0.0,795.0
7,rating,"(Tags.TARGET, Tags.REGRESSION)",int64,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,rating_binary,"(Tags.TARGET, Tags.BINARY_CLASSIFICATION)",int32,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,age,"(Tags.USER, Tags.CATEGORICAL)",int32,False,False,0.0,0.0,8.0,16.0,.//categories/unique.age.parquet,NaN,0.0,0.0,8.0


## Building the model

We now have everything we need to construct a model and train on our custom embeddings. In order to do so, we will leverage the `TensorInitializer` class and `Embeddings` function to set the `trainable` arg to `False`, so that our pre-trained embedddings will be frozen and not be updated during model training.

In [11]:
embed_dims = {}
embed_dims["movieId"] = pretrained_movie_embs.shape[1]

embeddings_init={
    "movieId": mm.TensorInitializer(pretrained_movie_embs),
}

embeddings_block = mm.Embeddings(
    train.schema.select_by_tag(Tags.CATEGORICAL),
    infer_embedding_sizes=True,
    embeddings_initializer=embeddings_init,
    trainable={'movieId': False},
    dim=embed_dims,
)
input_block = mm.InputBlockV2(train.schema, embeddings=embeddings_block)

Let us now feed our input_block into our model.

In [12]:
model = mm.DCNModel(
    train.schema,
    depth=2,
    input_block=input_block,
    deep_block=mm.MLPBlock([64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(target_column)
)

We could have created the model without passing the `embeddings_block` to the `input_block`. The model would still be able to infer how to construct itself (what should be the dimensionality of the input layer and so on) from the information contained in the schema.

However, passing a `TensorInitializer` into the constructor of the `input_block` tells our model to use our embedding table (`pretrained_movie_embs`) for that particular column of our dataset (`movieId`) as opposed to the model randomly initializing a brand new embedding matrix. For categorical columns we do not provide this information, the model will go with the standard initialization logic, which is to create an embedding table of appropriate size and perform random preinitialization.

Additionally, we set the `trainable` parameter for our pre-trained embeddings to `False` to ensure the embeddings will not be modified during training.

## Training

We train our model with `AUC` as our metric.

As we use synthetic data, the AUC score will not improve significantly.

In [13]:
%%time
opt = tf.keras.optimizers.Adagrad(learning_rate=1e-1)
model.compile(optimizer=opt, run_eagerly=False, metrics=[tf.keras.metrics.AUC()])

CPU times: user 13.2 ms, sys: 415 µs, total: 13.6 ms
Wall time: 12.1 ms


In [14]:
model.fit(train, batch_size=1024, epochs=5)

Epoch 1/5
98/98 [==============================] - 5s 11ms/step - loss: 0.6948 - auc: 0.5026 - regularization_loss: 0.0000e+00
Epoch 2/5
98/98 [==============================] - 1s 11ms/step - loss: 0.6933 - auc: 0.5035 - regularization_loss: 0.0000e+00
Epoch 3/5
98/98 [==============================] - 1s 11ms/step - loss: 0.6932 - auc: 0.5050 - regularization_loss: 0.0000e+00
Epoch 4/5
98/98 [==============================] - 1s 11ms/step - loss: 0.6931 - auc: 0.5072 - regularization_loss: 0.0000e+00
Epoch 5/5
98/98 [==============================] - 1s 11ms/step - loss: 0.6931 - auc: 0.5068 - regularization_loss: 0.0000e+00


The model trains using pretrained embeddings.